### Tyendinaga Data
##### Notes:
- data exported from 'Student' computer
- using data with filename Tyendinaga1_001.asc as main data source to make plots with
- Tyendinaga1_001.asc data was recorded on Oct. 26, 2025 from 18:07:12 to 18:27:12

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
import scipy.stats as ss
from sklearn.preprocessing import MinMaxScaler
import scipy.fft
from scipy.signal.windows import hann
from scipy import interpolate
from scipy.ndimage import uniform_filter
from scipy.signal import ShortTimeFFT
from scipy.signal import savgol_filter
from scipy.signal import hilbert
import subprocess
import os

In [ ]:
# list of the library versions used:
print("Version Notes:")
print("pandas:", pd.__version__)
#print("matplotlib.pyplot:", plt.__version__) # no version
print("numpy:", np.__version__)
print("scipy:", scipy.__version__)

##### DataFrame --> File Name Legend:
- df --> Tyendinaga_001.asc
- df2 --> Tyendinaga2_001.asc
- df3 --> Tyendinaga3_001.asc
- df4 --> Tyendinaga4_001.asc
- df5 --> Tyendinaga5_001.asc

In [ ]:
# Filename: Tyendinaga1
column_names = ['NS', 'EW', 'Z', 'nsL', 'ewL', 'zL', 'aY', 'aX', 'aZ']
df = pd.read_table('Tyendinaga1_001.asc', delimiter=r'\s+', encoding='latin-1', skiprows=32, names=column_names)
#df.head().style

In [ ]:
# Filename: Tyendinaga2
# Loading the rest of the Tyendinaga files so can make comparitive plots between datasets
column_names2 = ['NS', 'EW', 'Z', 'nsL', 'ewL', 'zL', 'aY', 'aX', 'aZ']
df2 = pd.read_table('Tyendinaga2_001.asc', delimiter=r'\s+', encoding='latin-1', skiprows=32, names=column_names2)
#df2.head().style

In [ ]:
# Filename: Tyendinaga3
column_names3 = ['NS', 'EW', 'Z', 'nsL', 'ewL', 'zL', 'aY', 'aX', 'aZ']
df3 = pd.read_table('Tyendinaga3_001.asc', delimiter=r'\s+', encoding='latin-1', skiprows=32, names=column_names3)
#df3.head().style

In [ ]:
# Filename: Tyendinaga4
column_names4 = ['NS', 'EW', 'Z', 'nsL', 'ewL', 'zL', 'aY', 'aX', 'aZ']
df4 = pd.read_table('Tyendinaga4_001.asc', delimiter=r'\s+', encoding='latin-1', skiprows=32, names=column_names4)
#df4.head().style

In [ ]:
# Filename: Tyendinaga5
column_names5 = ['NS', 'EW', 'Z', 'nsL', 'ewL', 'zL', 'aY', 'aX', 'aZ']
df5 = pd.read_table('Tyendinaga5_001.asc', delimiter=r'\s+', encoding='latin-1', skiprows=32, names=column_names5)
#df5.head().style

##### Check for null values:
- will show "False" if no null values

In [ ]:
any_null = df.isnull().any().any()
print("df: ",any_null) # Will show false if no null values
#df.info()
print("df2:", df2.isnull().any().any())
print("df3:", df3.isnull().any().any())
print("df4:", df4.isnull().any().any())
print("df5:", df5.isnull().any().any())

##### Columns of Interest:
- NS: North-South component of motion
- EW: East-West component of motion
- Z: Vertical (Z-axis) component of motion

##### Other Notes:
- Sampling rate for all files: 1024 Hz
- See below for preview of the data:

In [ ]:
#print(df.columns)
# Want to make a time column:
sampling_rate = 1024  # Hz (same for all data files)
dataframes = [df, df2, df3, df4, df5]
for i, dataframe in enumerate(dataframes):
    if i==0:
        print("dataframe: df")
    else:
        print(f"dataframe: df{i+1}")
    time = dataframe.index/sampling_rate
    dataframe['time (s)'] = time
    print(dataframe.head())
    print("")

***
### Plotting the Time Series
##### Notes:
- Plot the NS, EW, and Z data separately

In [ ]:
# reminder: dataframes = [df, df2, df3, df4, df5]
for i, dframe in enumerate(dataframes):
    if i==0:
        print('dataframe: df')
    else:
        print(f'dataframe: df{i+1}')

    # Plot:
    fig, axes = plt.subplots(3, 1, figsize=(10, 10))
    time_minutes = dframe['time (s)']/60
    # North-South data:
    axes[0].plot(time_minutes, dframe['NS'], label='North-South', color='forestgreen')
    axes[0].set_ylabel('North-South Component')
    axes[0].set_title('North-South Data')
    axes[0].set_xlabel('Time (min)')
    axes[0].grid(True)
    
    # East-West data:
    axes[1].plot(time_minutes, dframe['EW'], label='East-West', color='rebeccapurple')
    axes[1].set_ylabel('East-West Component')
    axes[1].set_title('East-West Data')
    axes[1].set_xlabel('Time (min)')
    axes[1].grid(True)
    
    # Z data:
    axes[2].plot(time_minutes, dframe['Z'], label='Z/Vertical Component', color='royalblue')
    axes[2].set_ylabel('Z Component')
    axes[2].set_title('Z-Component Data')
    axes[2].set_xlabel('Time (min)')
    axes[2].grid(True)
    
    plt.tight_layout()
    plt.show()

***
### Normalizing, Detrending and Shifting Data
##### Notes:
- data range of time series was normalized using MinMaxScaler from sklearn
- data was linearly detrended using signal.detrend from scipy
- the mean of the data was shifted to zero

In [ ]:
# Linearly detrend data, normalize and shift to zero mean:
# reminder: dataframes = [df, df2, df3, df4, df5]
for i, dframe in enumerate(dataframes):
    if i==0:
        print('dataframe: df')
    else:
        print(f'dataframe: df{i+1}')
        
    # Detrend
    dframe['NS_Detrended'] = signal.detrend(dframe['NS'], type='linear') # North-South data
    dframe['EW_Detrended'] = signal.detrend(dframe['EW'], type='linear') # East-West data
    dframe['Z_Detrended'] = signal.detrend(dframe['Z'], type='linear') # Z-component data

    # Normalize
    scaler = MinMaxScaler()
    dframe['NS_Normalized'] = scaler.fit_transform(dframe['NS_Detrended'].values.reshape(-1, 1)).flatten() # North-South
    dframe['EW_Normalized'] = scaler.fit_transform(dframe['EW_Detrended'].values.reshape(-1, 1)).flatten() # East-West
    dframe['Z_Normalized'] = scaler.fit_transform(dframe['Z_Detrended'].values.reshape(-1, 1)).flatten() # Z-component
    print(f"NS Min-Max normalized range: [{dframe['NS_Normalized'].min():.3f}, {dframe['NS_Normalized'].max():.3f}]")
    print(f"EW Min-Max normalized range: [{dframe['EW_Normalized'].min():.3f}, {dframe['EW_Normalized'].max():.3f}]")
    print(f"Z Min-Max normalized range: [{dframe['Z_Normalized'].min():.3f}, {dframe['Z_Normalized'].max():.3f}]")

    # Shift Mean:
    dframe['NS_Normalized_Shifted'] = (dframe['NS_Normalized'] - np.mean(dframe['NS_Normalized']))
    dframe['EW_Normalized_Shifted'] = (dframe['EW_Normalized'] - np.mean(dframe['EW_Normalized']))
    dframe['Z_Normalized_Shifted'] = (dframe['Z_Normalized'] - np.mean(dframe['Z_Normalized']))
    print(f"New range for NS: [{dframe['NS_Normalized_Shifted'].min():.3f}, {dframe['NS_Normalized_Shifted'].max():.3f}]")
    print(f"New mean for NS: {dframe['NS_Normalized_Shifted'].mean():.6f}")
    print(f"New range for EW: [{dframe['EW_Normalized_Shifted'].min():.3f}, {dframe['EW_Normalized_Shifted'].max():.3f}]")
    print(f"New mean for EW: {dframe['EW_Normalized_Shifted'].mean():.6f}")
    print(f"New range for Z: [{dframe['Z_Normalized_Shifted'].min():.3f}, {dframe['Z_Normalized_Shifted'].max():.3f}]")
    print(f"New mean for Z: {dframe['Z_Normalized_Shifted'].mean():.6f}")

    # Plot:
    fig, axes = plt.subplots(3, 1, figsize=(10, 10))
    time_minutes = dframe['time (s)']/60
    # North-South data:
    axes[0].plot(time_minutes, dframe['NS_Normalized_Shifted'], label='North-South', color='forestgreen')
    axes[0].set_ylabel('North-South Component')
    axes[0].set_title('DNS North-South Data')
    axes[0].set_xlabel('Time (min)')
    axes[0].grid(True)

    # East-West data:
    axes[1].plot(time_minutes, dframe['EW_Normalized_Shifted'], label='East-West', color='rebeccapurple')
    axes[1].set_ylabel('East-West Component')
    axes[1].set_title('DNS East-West Data')
    axes[1].set_xlabel('Time (min)')
    axes[1].grid(True)
    
    # Z data:
    axes[2].plot(time_minutes, dframe['Z_Normalized_Shifted'], label='Z/Vertical Component', color='royalblue')
    axes[2].set_ylabel('Z Component')
    axes[2].set_title('DNS Z-Component Data')
    axes[2].set_xlabel('Time (min)')
    axes[2].grid(True)
    
    plt.tight_layout()
    plt.show()
    print("DNS = Detrended, Normalized, Shifted")
    print("")
    # Stats for plots
    print("North-South Stats:")
    print(ss.describe(dframe['NS_Normalized_Shifted']))
    print("median:", dframe["NS_Normalized_Shifted"].median())
    print("mode:", ss.mode(df["NS_Normalized_Shifted"])) 
    print("standard deviation:", dframe["NS_Normalized_Shifted"].std())
    print("")
    print("East-West Stats:")
    print(ss.describe(dframe['EW_Normalized_Shifted']))
    print("median:", dframe["EW_Normalized_Shifted"].median())
    print("mode:", ss.mode(dframe["EW_Normalized_Shifted"])) 
    print("standard deviation:", dframe["EW_Normalized_Shifted"].std())
    print("")
    print("Z-component Stats:")
    print(ss.describe(dframe['Z_Normalized_Shifted']))
    print("median:", dframe["Z_Normalized_Shifted"].median())
    print("mode:", ss.mode(dframe["Z_Normalized_Shifted"])) 
    print("standard deviation:", dframe["Z_Normalized_Shifted"].std())
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

***
### Subsampling the Time Series
##### Notes:
- the original sampling rate for the data is 1024 Hz
- different step sizes were applied to the normalized, detrended and mean-shifted (DNS) data for subsampling
- adjust x-axis limits to focus in on certain times

#### "percent_check" Function for Choosing Subsampling Step Size:
##### Notes:
- this method uses interpolation to compare subsampled data to the full DNS data
- the absolute error between the full DNS data and the reconstructed subsampled data is calculated and divided by the full data range
- this result is then used determine if the error is within the selected threshold (i.e. 5%)

In [ ]:
# First use percent_check function to choose step-size (i.e. decimation_factor) to use for subsample function
def percent_check(data, column, step, tolerance=0.05):
    """Determine if error for subsampled data is within 5% tolerance compared to full DNS data range
    data: dataframe to use
    column: column of interest for y values
    step: data step to use for subsampling
    tolerance: error tolerance expressed as a decimal"""
    # Get original series
    x_original = np.arange(len(data))
    y_original = data[column].values
    
    # Get subsampled series
    x_subsampled = np.arange(0, len(data), step)
    y_subsampled = data[column].iloc[::step].values
    
    # Interpolate subsampled data back to original timestamps
    f_interp = interpolate.interp1d(x_subsampled, y_subsampled, kind='linear', fill_value='extrapolate')
    y_reconstructed = f_interp(x_original)
    
    # Calculate absolute errors
    abs_errors = np.abs(y_original - y_reconstructed)
    
    # Calculate data range (the scale of the data)
    data_range = np.max(y_original) - np.min(y_original)
    #print("data_range:", data_range)
    # Normalize errors by data range instead of individual values
    normalized_errors = abs_errors / data_range
    
    # Test if within tolerance
    within_tolerance = normalized_errors <= tolerance
    pct_within = (np.sum(within_tolerance) / len(within_tolerance)) * 100
    
    # Overall pass/fail
    passes = pct_within >= 95  # At least 95% of points within tolerance
    
    results = {'step': step,
        'tolerance': f'{tolerance*100}%',
        'passes': passes,
        'points_within_tolerance': f'{pct_within: .4f}%',
        'num_original_points': len(y_original),
        'num_subsampled_points': len(y_subsampled),
        'data_range': f'{data_range: .4f}',  
        'max_normalized_error': f'{(np.max(normalized_errors)*100): .4f}%' }
    
    return results
# step_1 = 12
# Results for df:
NS_results_step_1 = percent_check(df, 'NS_Normalized_Shifted', 7, tolerance=0.05)
print("NS results 1 (df):", NS_results_step_1)
EW_results_step_1 = percent_check(df, 'EW_Normalized_Shifted', 7, tolerance=0.05)
print("EW results 1 (df):", EW_results_step_1)
Z_results_step_1 = percent_check(df, 'Z_Normalized_Shifted', 7, tolerance=0.05)
print("Z results 1 (df):", Z_results_step_1)
print("")
# Results for df2:
NS_results_step_1_df2 = percent_check(df2, 'NS_Normalized_Shifted', 7, tolerance=0.05)
print("NS results 1 (df2):", NS_results_step_1_df2)
EW_results_step_1_df2 = percent_check(df2, 'EW_Normalized_Shifted', 7, tolerance=0.05)
print("EW results 1 (df2):", EW_results_step_1_df2)
Z_results_step_1_df2 = percent_check(df2, 'Z_Normalized_Shifted', 7, tolerance=0.05)
print("Z results 1 (df2):", Z_results_step_1_df2)
print("")
# Results for df3:
NS_results_step_1_df3 = percent_check(df3, 'NS_Normalized_Shifted', 7, tolerance=0.05)
print("NS results 1 (df3):", NS_results_step_1_df3)
EW_results_step_1_df3 = percent_check(df3, 'EW_Normalized_Shifted', 7, tolerance=0.05)
print("EW results 1 (df3):", EW_results_step_1_df3)
Z_results_step_1_df3 = percent_check(df3, 'Z_Normalized_Shifted', 7, tolerance=0.05)
print("Z results 1 (df3):", Z_results_step_1_df3)
print("")
# Results for df4:
NS_results_step_1_df4 = percent_check(df4, 'NS_Normalized_Shifted', 7, tolerance=0.05)
print("NS results 1 (df4):", NS_results_step_1_df4)
EW_results_step_1_df4 = percent_check(df4, 'EW_Normalized_Shifted', 7, tolerance=0.05) # passes with stepsizeof 10 but not 12
print("EW results 1 (df4):", EW_results_step_1_df4)
Z_results_step_1_df4 = percent_check(df4, 'Z_Normalized_Shifted', 7, tolerance=0.05) # Z passes check with stepsize of 12 but not 10
print("Z results 1 (df4):", Z_results_step_1_df4)
print("")
# Results for df5:
NS_results_step_1_df5 = percent_check(df5, 'NS_Normalized_Shifted', 7, tolerance=0.05)
print("NS results 1 (df5):", NS_results_step_1_df5)
EW_results_step_1_df5 = percent_check(df5, 'EW_Normalized_Shifted', 7, tolerance=0.05)
print("EW results 1 (df5):", EW_results_step_1_df5)
Z_results_step_1_df5 = percent_check(df5, 'Z_Normalized_Shifted', 7, tolerance=0.05)
print("Z results 1 (df5):", Z_results_step_1_df5)
print("")
# Note: 7 is the largest stepsize that passes the tolerance check everywhere (i.e. for all dataframes), simultaneously

#### percent_check results:
- A step size of 7 was the largest that passed the percent_check test for NS, EW, and Z data simultaneously for all dataframes (plotted below)

In [ ]:
# plotting time series using percent_check step sizes to compare to full DNS plots
for i, dframe in enumerate(dataframes):
    if i==0:
        print('dataframe: df')
    else:
        print(f'dataframe: df{i+1}')
    # Plot:
    fig, axes = plt.subplots(3, 1, figsize=(10, 10))
    time_minutes = dframe['time (s)']/60
    # North-South data:
    axes[0].plot(time_minutes, dframe['NS_Normalized_Shifted'], label='North-South (Full)', color='red')
    axes[0].plot(time_minutes[::7], dframe['NS_Normalized_Shifted'][::7], label='North-South (Step=7)', color='forestgreen')
    axes[0].set_ylabel('North-South Component')
    axes[0].set_title('Subsampled and Full DNS North-South Data')
    axes[0].set_xlabel('Time (min)')
    axes[0].legend()
    axes[0].grid(True)
    
    # East-West data:
    axes[1].plot(time_minutes, dframe['EW_Normalized_Shifted'], label='East-West (Full)', color='red')
    axes[1].plot(time_minutes[::7], dframe['EW_Normalized_Shifted'][::7], label='East-West (Step=7)', color='rebeccapurple')
    axes[1].set_ylabel('East-West Component')
    axes[1].set_title('Subsampled and Full DNS East-West Data')
    axes[1].set_xlabel('Time (min)')
    axes[1].legend()
    axes[1].grid(True)
    
    # Z data:
    axes[2].plot(time_minutes, dframe['Z_Normalized_Shifted'], label='Z (Full)', color='red')
    axes[2].plot(time_minutes[::7], dframe['Z_Normalized_Shifted'][::7], label='Z (Step=7)', color='royalblue')
    axes[2].set_ylabel('Z Component')
    axes[2].set_title('Subsampled and Full DNS Z-Component Data')
    axes[2].set_xlabel('Time (min)')
    axes[2].legend()
    axes[2].grid(True)
    
    plt.tight_layout()
    plt.show()

#### percent_check function using scipy.signal.decimate for y_subsampled instead of indexing:
- testing a different method for subsampling with the same step size (i.e. 7)

In [ ]:
# Try to use percent_check for scipy decimate version of subsampling
def percent_check_scipy_decimate(data, column, step, tolerance=0.05):
    x_original = np.arange(len(data))
    y_original = data[column].values
    
    # Get subsampled series
    x_subsampled = np.arange(0, len(data), step)
    y_subsampled=scipy.signal.decimate(data[column].values, step, ftype='iir', zero_phase=True)
    
    # Interpolate subsampled data back to original timestamps
    f_interp = interpolate.interp1d(x_subsampled, y_subsampled, kind='linear', fill_value='extrapolate')
    y_reconstructed = f_interp(x_original)
    
    # Calculate absolute errors
    abs_errors = np.abs(y_original - y_reconstructed)
    
    # Calculate data range (the scale of the data)
    data_range = np.max(y_original) - np.min(y_original)
    #print("data_range:", data_range)
    # Normalize errors by data range instead of individual values
    normalized_errors = abs_errors / data_range
    
    # Test if within tolerance
    within_tolerance = normalized_errors <= tolerance
    pct_within = (np.sum(within_tolerance) / len(within_tolerance)) * 100
    
    # Overall pass/fail
    passes = pct_within >= 95  # At least 95% of points within tolerance
    
    results = {'step': step,
        'tolerance': f'{tolerance*100}%',
        'passes': passes,
        'points_within_tolerance': f'{pct_within: .4f}%',
        'num_original_points': len(y_original),
        'num_subsampled_points': len(y_subsampled),
        'data_range': f'{data_range: .4f}',  
        'max_normalized_error': f'{(np.max(normalized_errors)*100): .4f}%' }
    return results

# results for df:
NS_results_test_1_df = percent_check_scipy_decimate(df, 'NS_Normalized_Shifted', 7, tolerance=0.05)
print("NS_results_test_1 (df):", NS_results_test_1_df)
EW_results_test_1_df = percent_check_scipy_decimate(df, 'EW_Normalized_Shifted', 7, tolerance=0.05)
print("EW_results_test_1 (df):", EW_results_test_1_df)
Z_results_test_1_df = percent_check_scipy_decimate(df, 'Z_Normalized_Shifted', 7, tolerance=0.05)
print("Z_results_test_1 (df):", Z_results_test_1_df)
print("")
# results for df2:
NS_results_test_1_df2 = percent_check_scipy_decimate(df2, 'NS_Normalized_Shifted', 7, tolerance=0.05)
print("NS_results_test_1 (df2):", NS_results_test_1_df2)
EW_results_test_1_df2 = percent_check_scipy_decimate(df2, 'EW_Normalized_Shifted', 7, tolerance=0.05)
print("EW_results_test_1 (df2):", EW_results_test_1_df2)
Z_results_test_1_df2 = percent_check_scipy_decimate(df2, 'Z_Normalized_Shifted', 7, tolerance=0.05)
print("Z_results_test_1 (df2):", Z_results_test_1_df2)
print("")
# results for df3:
NS_results_test_1_df3 = percent_check_scipy_decimate(df3, 'NS_Normalized_Shifted', 7, tolerance=0.05)
print("NS_results_test_1 (df3):", NS_results_test_1_df3)
EW_results_test_1_df3 = percent_check_scipy_decimate(df3, 'EW_Normalized_Shifted', 7, tolerance=0.05)
print("EW_results_test_1 (df3):", EW_results_test_1_df3)
Z_results_test_1_df3 = percent_check_scipy_decimate(df3, 'Z_Normalized_Shifted', 7, tolerance=0.05)
print("Z_results_test_1 (df3):", Z_results_test_1_df3)
print("")
# results for df4:
NS_results_test_1_df4 = percent_check_scipy_decimate(df4, 'NS_Normalized_Shifted', 7, tolerance=0.05)
print("NS_results_test_1 (df4):", NS_results_test_1_df4)
EW_results_test_1_df4 = percent_check_scipy_decimate(df4, 'EW_Normalized_Shifted', 7, tolerance=0.05)
print("EW_results_test_1 (df4):", EW_results_test_1_df4)
Z_results_test_1_df4 = percent_check_scipy_decimate(df4, 'Z_Normalized_Shifted', 7, tolerance=0.05)
print("Z_results_test_1 (df4):", Z_results_test_1_df4)
print("")
# results for df5:
NS_results_test_1_df5 = percent_check_scipy_decimate(df5, 'NS_Normalized_Shifted', 7, tolerance=0.05)
print("NS_results_test_1 (df5):", NS_results_test_1_df5)
EW_results_test_1_df5 = percent_check_scipy_decimate(df5, 'EW_Normalized_Shifted', 7, tolerance=0.05)
print("EW_results_test_1 (df5):", EW_results_test_1_df5)
Z_results_test_1_df5 = percent_check_scipy_decimate(df5, 'Z_Normalized_Shifted', 7, tolerance=0.05)
print("Z_results_test_1 (df5):", Z_results_test_1_df5)
print("")

In [ ]:
# try using scipy.signal.decimate and compare to indexing results
original_sampling_rate = 1024 # Hz
def subsample(data, decimation_factor, sr):
    """Subsample data using scipy.signal.decimate with anti-aliasing filter.
    data: input data (array)
    decimation_factor: factor by which to reduce sampling rate
    sr: Original sampling rate"""
    # scipy.signal.decimate applies an anti-aliasing filter automatically
    # Use a higher order filter for better anti-aliasing (default is 8)
    decimated_data = scipy.signal.decimate(data, decimation_factor, ftype='iir', zero_phase=True)  
    # get new sampling rate:
    new_sr = sr / decimation_factor
    #print(f"New sampling rate: {new_sr} Hz")
    return decimated_data, new_sr

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Compute results for NS, EW, and Z data:
# (Using stepsize of 7 for decimation_factor)
# df:
NS_sub_results_df, NS_new_sr_df = subsample(df['NS_Normalized_Shifted'].values, 7, original_sampling_rate)
EW_sub_results_df, EW_new_sr_df = subsample(df['EW_Normalized_Shifted'].values, 7, original_sampling_rate)
Z_sub_results_df, Z_new_sr_df = subsample(df['Z_Normalized_Shifted'].values, 7, original_sampling_rate)
#df2:
NS_sub_results_df2, NS_new_sr_df2 = subsample(df2['NS_Normalized_Shifted'].values, 7, original_sampling_rate)
EW_sub_results_df2, EW_new_sr_df2 = subsample(df2['EW_Normalized_Shifted'].values, 7, original_sampling_rate)
Z_sub_results_df2, Z_new_sr_df2 = subsample(df2['Z_Normalized_Shifted'].values, 7, original_sampling_rate)
#df3:
NS_sub_results_df3, NS_new_sr_df3 = subsample(df3['NS_Normalized_Shifted'].values, 7, original_sampling_rate)
EW_sub_results_df3, EW_new_sr_df3 = subsample(df3['EW_Normalized_Shifted'].values, 7, original_sampling_rate)
Z_sub_results_df3, Z_new_sr_df3 = subsample(df3['Z_Normalized_Shifted'].values, 7, original_sampling_rate)
#df4:
NS_sub_results_df4, NS_new_sr_df4 = subsample(df4['NS_Normalized_Shifted'].values, 7, original_sampling_rate)
EW_sub_results_df4, EW_new_sr_df4 = subsample(df4['EW_Normalized_Shifted'].values, 7, original_sampling_rate)
Z_sub_results_df4, Z_new_sr_df4 = subsample(df4['Z_Normalized_Shifted'].values, 7, original_sampling_rate)
#df5:
NS_sub_results_df5, NS_new_sr_df5 = subsample(df5['NS_Normalized_Shifted'].values, 7, original_sampling_rate)
EW_sub_results_df5, EW_new_sr_df5 = subsample(df5['EW_Normalized_Shifted'].values, 7, original_sampling_rate)
Z_sub_results_df5, Z_new_sr_df5 = subsample(df5['Z_Normalized_Shifted'].values, 7, original_sampling_rate)
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
NS_subsampled_data = [NS_sub_results_df, NS_sub_results_df2, NS_sub_results_df3, NS_sub_results_df4, NS_sub_results_df5]
EW_subsampled_data = [EW_sub_results_df, EW_sub_results_df2, EW_sub_results_df3, EW_sub_results_df4, EW_sub_results_df5]
Z_subsampled_data = [Z_sub_results_df, Z_sub_results_df2, Z_sub_results_df3, Z_sub_results_df4, Z_sub_results_df5]

for i, dframe in enumerate(dataframes):
    # Plot:
    time_minutes = dframe['time (s)']/60
    # North-South data:
    plt.figure(figsize=(12,4))
    plt.plot(time_minutes, dframe['NS_Normalized_Shifted'], label='North-South (Full)', color='red')
    plt.plot(time_minutes[::7], NS_subsampled_data[i], label='North-South (Step=7)', color='forestgreen')
    plt.ylabel('North-South Component')
    plt.title('Subsampled and Full DNS North-South Data')
    plt.xlabel('Time (min)')
    plt.legend(loc='upper left')
    plt.grid(True)
    
    # East-West data:
    plt.figure(figsize=(12,4))
    plt.plot(time_minutes, dframe['EW_Normalized_Shifted'], label='East-West (Full)', color='red')
    plt.plot(time_minutes[::7], EW_subsampled_data[i], label='East-West (Step=7)', color='rebeccapurple')
    plt.ylabel('East-West Component')
    plt.title('Subsampled and Full DNS East-West Data')
    plt.xlabel('Time (min)')
    plt.legend(loc='upper left')
    plt.grid(True)
    
    # Z data:
    plt.figure(figsize=(12,4))
    plt.plot(time_minutes, dframe['Z_Normalized_Shifted'], label='Z (Full)', color='red')
    plt.plot(time_minutes[::7], Z_subsampled_data[i], label='Z (Step=7)', color='royalblue')
    plt.ylabel('Z Component')
    plt.title('Subsampled and Full DNS Z-Component Data')
    plt.xlabel('Time (min)')
    plt.legend(loc='upper left')
    plt.grid(True)

plt.tight_layout()
plt.show()
# Note: New sampling rate is 146.285714... Hz using decimation_factor of 7

#### Comparing percent_check results from scipy.signal.decimate and indexing for y_subsampled
- There were a similar percentage of points within the error tolerance when using the scipy.signal.decimate method compared to the indexing method for subsampling
- For later cells, the scipy.signal.decimate method will be used as it includes a built-in anti-aliasing filter

***
### Tapering the Time Series
##### Notes:
- a Hann window/raised cosine was used to taper the DNS time series
- the data is tapered starting at 10% from the edges (this percentage can be adjusted)

In [ ]:
# reminder (defined in previous cell): dataframes = [df, df2, df3, df4, df5]
def apply_hann_window(data, NS_column, EW_column, Z_column, taper_fraction = 0.1):
    """Taper edges of time series with Hann window
    taper_fraction = 0.1 will start taper at 10% from edges"""
    n= len(data)
    taper_length = int(n * taper_fraction) # should be same for all components

    # NS Data:
    # Create a copy of the data
    NS_windowed_data = data[NS_column].copy()
    NS_windowed_data = NS_windowed_data.astype(float) # needed for FFT code to work properly later on
    # Apply Hann window to the left edge
    NS_left_window = np.hanning(2 * taper_length)[:taper_length]
    NS_windowed_data.iloc[:taper_length] *= NS_left_window
    # Apply Hann window to the right edge
    NS_right_window = np.hanning(2 * taper_length)[taper_length:]
    NS_windowed_data.iloc[-taper_length:] *= NS_right_window

    # EW Data:
    # Create a copy of the data
    EW_windowed_data = data[EW_column].copy()
    EW_windowed_data = EW_windowed_data.astype(float) 
    # Apply Hann window to the left edge
    EW_left_window = np.hanning(2 * taper_length)[:taper_length]
    EW_windowed_data.iloc[:taper_length] *= EW_left_window
    # Apply Hann window to the right edge
    EW_right_window = np.hanning(2 * taper_length)[taper_length:]
    EW_windowed_data.iloc[-taper_length:] *= EW_right_window

    # Z Data:
    Z_windowed_data = data[Z_column].copy()
    Z_windowed_data = Z_windowed_data.astype(float) 
    # Apply Hann window to the left edge
    Z_left_window = np.hanning(2 * taper_length)[:taper_length]
    Z_windowed_data.iloc[:taper_length] *= Z_left_window
    # Apply Hann window to the right edge
    Z_right_window = np.hanning(2 * taper_length)[taper_length:]
    Z_windowed_data.iloc[-taper_length:] *= Z_right_window
    
    return NS_windowed_data, EW_windowed_data, Z_windowed_data
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# df:
NS_hann_result_df, EW_hann_result_df, Z_hann_result_df = apply_hann_window(df, 'NS_Normalized_Shifted', 'EW_Normalized_Shifted', 'Z_Normalized_Shifted')
# df2:
NS_hann_result_df2, EW_hann_result_df2, Z_hann_result_df2 = apply_hann_window(df2, 'NS_Normalized_Shifted', 'EW_Normalized_Shifted', 'Z_Normalized_Shifted')
# df3:
NS_hann_result_df3, EW_hann_result_df3, Z_hann_result_df3 = apply_hann_window(df3, 'NS_Normalized_Shifted', 'EW_Normalized_Shifted', 'Z_Normalized_Shifted')
# df4:
NS_hann_result_df4, EW_hann_result_df4, Z_hann_result_df4 = apply_hann_window(df4, 'NS_Normalized_Shifted', 'EW_Normalized_Shifted', 'Z_Normalized_Shifted')
# df5:
NS_hann_result_df5, EW_hann_result_df5, Z_hann_result_df5 = apply_hann_window(df5, 'NS_Normalized_Shifted', 'EW_Normalized_Shifted', 'Z_Normalized_Shifted')
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
NS_hann_results = [NS_hann_result_df, NS_hann_result_df2, NS_hann_result_df3, NS_hann_result_df4, NS_hann_result_df5]
EW_hann_results = [EW_hann_result_df, EW_hann_result_df2, EW_hann_result_df3, EW_hann_result_df4, EW_hann_result_df5]
Z_hann_results = [Z_hann_result_df, Z_hann_result_df2, Z_hann_result_df3, Z_hann_result_df4, Z_hann_result_df5]

# Plot tapered data with full data for all dataframes:
for i, dframe in enumerate(dataframes):
    if i == 0:
        print("dataframe: df")
    else:
        print(f"dataframe: df{i+1}")
    # Plot:
    fig, axes = plt.subplots(3, 1, figsize=(10, 10))
    time_minutes = dframe['time (s)']/60
    # North-South data:
    axes[0].plot(time_minutes, dframe['NS_Normalized_Shifted'], label='North-South (Full)', color='red')
    axes[0].plot(time_minutes, NS_hann_results[i], label='North-South (Tapered)', color='forestgreen')
    axes[0].set_ylabel('North-South Component')
    axes[0].set_title('Tapered and Full DNS North-South Data')
    axes[0].set_xlabel('Time (min)')
    axes[0].legend()
    axes[0].grid(True)
    
    # East-West data:
    axes[1].plot(time_minutes, dframe['EW_Normalized_Shifted'], label='East-West (Full)', color='red')
    axes[1].plot(time_minutes, EW_hann_results[i], label='East-West (Tapered)', color='rebeccapurple')
    axes[1].set_ylabel('East-West Component')
    axes[1].set_title('Tapered and Full DNS East-West Data')
    axes[1].set_xlabel('Time (min)')
    axes[1].legend()
    axes[1].grid(True)
    
    # Z data:
    axes[2].plot(time_minutes, dframe['Z_Normalized_Shifted'], label='Z (Full)', color='red')
    axes[2].plot(time_minutes, Z_hann_results[i], label='Z (Tapered)', color='royalblue')
    axes[2].set_ylabel('Z Component')
    axes[2].set_title('Tapered and Full DNS Z-Component Data')
    axes[2].set_xlabel('Time (min)')
    axes[2].legend()
    #axes[2].set_xlim(-0.001,0.01)
    axes[2].grid(True)
    
    plt.tight_layout()
    plt.show()

#### 'percent_check_tapered' for tapered time series:
- a tapering version of the 'percent_check' function that was used for subsampling
- based on tapering starting at 10% from the data edges

In [ ]:
def percent_check_tapered(original_data, tapered_data_full, padding, tolerance=0.05):
    """original_data : Original unpadded time series (array)
    tapered_data_full : Tapered time series, includes padding (array)
    padding : Amount of padding added to each end (int)
    tolerance : Acceptable error as fraction of data range (float)"""
    # Extract original from tapered data (remove padding)
    if padding > 0:
        tapered_original_region = tapered_data_full[padding:-padding]
    else:
        tapered_original_region = tapered_data_full
    
    # Direct comparison: tapered vs original
    abs_errors = np.abs(original_data - tapered_original_region)
    # normalization
    data_range = np.max(original_data) - np.min(original_data)
    normalized_errors = abs_errors / data_range
    
    # Calculate metrics
    within_tolerance = normalized_errors <= tolerance
    pct_within = (np.sum(within_tolerance) / len(within_tolerance))*100
    
    # Mean Absolute Error (MAE) metrics
    mae = np.mean(abs_errors)
    std_original = np.std(original_data)
    mae_pct_std = (mae / std_original)*100
    
    passes = pct_within >= 95
    
    results = {
        'padding': padding,
        'tolerance': f'{tolerance*100}%',
        'passes': passes,
        'points_within_tolerance': f'{pct_within: .4f}%',
        'max_normalized_error': f'{(np.max(normalized_errors)*100): .4f}%',
        'mean_abs_error': f'{mae: .4f}',
        'mae_as_%_of_std': f'{mae_pct_std: .4f}%',
        'num_points': len(original_data),
        'data_range': f'{data_range: .4f}'}
    
    return results
# df:
NS_taper_results_1_df = percent_check_tapered(df['NS_Normalized_Shifted'], NS_hann_result_df, padding=0, tolerance=0.05)
print("NS_taper_results_1 (df):", NS_taper_results_1_df)
EW_taper_results_1_df = percent_check_tapered(df['EW_Normalized_Shifted'], EW_hann_result_df, padding=0, tolerance=0.05)
print("EW_taper_results_1 (df):", EW_taper_results_1_df)
Z_taper_results_1_df = percent_check_tapered(df['Z_Normalized_Shifted'], Z_hann_result_df, padding=0, tolerance=0.05)
print("Z_taper_results_1 (df):", Z_taper_results_1_df)
print("")
# df2:
NS_taper_results_1_df2 = percent_check_tapered(df2['NS_Normalized_Shifted'], NS_hann_result_df2, padding=0, tolerance=0.05)
print("NS_taper_results_1 (df2):", NS_taper_results_1_df2)
EW_taper_results_1_df2 = percent_check_tapered(df2['EW_Normalized_Shifted'], EW_hann_result_df2, padding=0, tolerance=0.05)
print("EW_taper_results_1 (df2):", EW_taper_results_1_df2)
Z_taper_results_1_df2 = percent_check_tapered(df2['Z_Normalized_Shifted'], Z_hann_result_df2, padding=0, tolerance=0.05)
print("Z_taper_results_1 (df2):", Z_taper_results_1_df2)
print("")
# df3:
NS_taper_results_1_df3 = percent_check_tapered(df3['NS_Normalized_Shifted'], NS_hann_result_df3, padding=0, tolerance=0.05)
print("NS_taper_results_1 (df3):", NS_taper_results_1_df3)
EW_taper_results_1_df3 = percent_check_tapered(df3['EW_Normalized_Shifted'], EW_hann_result_df3, padding=0, tolerance=0.05)
print("EW_taper_results_1 (df3):", EW_taper_results_1_df3)
Z_taper_results_1_df3 = percent_check_tapered(df3['Z_Normalized_Shifted'], Z_hann_result_df3, padding=0, tolerance=0.05)
print("Z_taper_results_1 (df3):", Z_taper_results_1_df3)
print("")
# df4:
NS_taper_results_1_df4 = percent_check_tapered(df4['NS_Normalized_Shifted'], NS_hann_result_df4, padding=0, tolerance=0.05)
print("NS_taper_results_1 (df4):", NS_taper_results_1_df4)
EW_taper_results_1_df4 = percent_check_tapered(df4['EW_Normalized_Shifted'], EW_hann_result_df4, padding=0, tolerance=0.05)
print("EW_taper_results_1 (df4):", EW_taper_results_1_df4)
Z_taper_results_1_df4 = percent_check_tapered(df4['Z_Normalized_Shifted'], Z_hann_result_df4, padding=0, tolerance=0.05)
print("Z_taper_results_1 (df4):", Z_taper_results_1_df4)
print("")
# df5:
NS_taper_results_1_df5 = percent_check_tapered(df5['NS_Normalized_Shifted'], NS_hann_result_df5, padding=0, tolerance=0.05)
print("NS_taper_results_1 (df5):", NS_taper_results_1_df5)
EW_taper_results_1_df5 = percent_check_tapered(df5['EW_Normalized_Shifted'], EW_hann_result_df5, padding=0, tolerance=0.05)
print("EW_taper_results_1 (df5):", EW_taper_results_1_df5)
Z_taper_results_1_df5 = percent_check_tapered(df5['Z_Normalized_Shifted'], Z_hann_result_df5, padding=0, tolerance=0.05)
print("Z_taper_results_1 (df5):", Z_taper_results_1_df5)

***
### Combining Subsampling and Tapering
##### Notes:
- The full series was tapered before subsampling
- A Hann window was used for tapering
- A step size of 7 was used for subsampling, consistent across the North-South, East-West, and Up-Down (Z) data
    - adjust the step size used for subsampling as needed

In [ ]:
# Put tapered results into subsample function from a previous cell:
def subsample_windowed(NS_windowed, EW_windowed, Z_windowed, sr=1024, decimation_factor=7):
    NS_decimated_data = scipy.signal.decimate(NS_windowed, decimation_factor, ftype='iir', zero_phase=True)
    EW_decimated_data = scipy.signal.decimate(EW_windowed, decimation_factor, ftype='iir', zero_phase=True)
    Z_decimated_data = scipy.signal.decimate(Z_windowed, decimation_factor, ftype='iir', zero_phase=True)
    new_sr = sr/decimation_factor
    return NS_decimated_data, EW_decimated_data, Z_decimated_data, new_sr
# Results for all dataframes:  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# df:
NS_decimated_data_df, EW_decimated_data_df, Z_decimated_data_df, new_sr_df = subsample_windowed(NS_hann_result_df, EW_hann_result_df, Z_hann_result_df)
# df2:
NS_decimated_data_df2, EW_decimated_data_df2, Z_decimated_data_df2, new_sr_df2 = subsample_windowed(NS_hann_result_df2, EW_hann_result_df2, Z_hann_result_df2)
# df3:
NS_decimated_data_df3, EW_decimated_data_df3, Z_decimated_data_df3, new_sr_df3 = subsample_windowed(NS_hann_result_df3, EW_hann_result_df3, Z_hann_result_df3)
# df4:
NS_decimated_data_df4, EW_decimated_data_df4, Z_decimated_data_df4, new_sr_df4 = subsample_windowed(NS_hann_result_df4, EW_hann_result_df4, Z_hann_result_df4)
# df5:
NS_decimated_data_df5, EW_decimated_data_df5, Z_decimated_data_df5, new_sr_df5 = subsample_windowed(NS_hann_result_df5, EW_hann_result_df5, Z_hann_result_df5)
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
NS_taper_sub = [NS_decimated_data_df, NS_decimated_data_df2, NS_decimated_data_df3, NS_decimated_data_df4, NS_decimated_data_df5]
EW_taper_sub = [EW_decimated_data_df, EW_decimated_data_df2, EW_decimated_data_df3, EW_decimated_data_df4, EW_decimated_data_df5]
Z_taper_sub = [Z_decimated_data_df, Z_decimated_data_df2, Z_decimated_data_df3, Z_decimated_data_df4, Z_decimated_data_df5]

for i, dframe in enumerate(dataframes):
    if i ==0:
        print("dataframe: df")
    else:
        print(f"dataframe: df{i+1}")
    # Plot:
    fig, axes = plt.subplots(3, 1, figsize=(12, 12))
    time_minutes = dframe['time (s)']/60
    # North-South data:
    axes[0].plot(time_minutes, dframe['NS_Normalized_Shifted'], label='North-South (Full)', color='red')
    axes[0].plot(time_minutes[::7], NS_taper_sub[i], label='Tapered North-South (Step=7)', color='forestgreen') # make sure step for time_minutes=decimation_factor
    axes[0].set_ylabel('North-South Component')
    axes[0].set_title('Tapered/Subsampled and Full DNS North-South Data')
    axes[0].set_xlabel('Time (min)')
    axes[0].legend()
    axes[0].grid(True)
    
    # East-West data:
    axes[1].plot(time_minutes, dframe['EW_Normalized_Shifted'], label='East-West (Full)', color='red')
    axes[1].plot(time_minutes[::7], EW_taper_sub[i], label='Tapered East-West (Step=7)', color='rebeccapurple')
    axes[1].set_ylabel('East-West Component')
    axes[1].set_title('Tapered/Subsampled and Full DNS East-West Data')
    axes[1].set_xlabel('Time (min)')
    axes[1].legend()
    axes[1].grid(True)
    
    # Z data:
    axes[2].plot(time_minutes, dframe['Z_Normalized_Shifted'], label='Z (Full)', color='red')
    axes[2].plot(time_minutes[::7], Z_taper_sub[i], label='Tapered Z (Step=7)', color='royalblue')
    axes[2].set_ylabel('Z Component')
    axes[2].set_title('Tapered/Subsampled and Full DNS Z-Component Data')
    axes[2].set_xlabel('Time (min)')
    axes[2].legend()
    #axes[2].set_xlim(-0.001,0.01)
    axes[2].grid(True)
    
    plt.tight_layout()
    plt.show()

***
### Fast Fourier Transform
##### Notes:
- Applying the fast fourier transform to subsampled and tapered DNS data and plotting the resulting magnitude and phase spectra
    - (commented out phase spectra for now)
    - also defined a coherent gain function in attempt to correct for the amplitude reduction from windowing<br>
<br> Effects of subsampling/tapering:
    - removes higher frequency content
    - reduces spectral leakage caused by discontinuities

In [ ]:
# Hann window one column at a time:
def apply_hann_window1(data, column, taper_fraction = 0.1):
    """Taper edges of time series with Hann window
    taper_fraction = 0.1 will start taper at 10% from edges"""
    n= len(data)
    taper_length = int(n * taper_fraction)
    
    # Create a copy of the data
    windowed_data = data[column].copy()
    windowed_data = windowed_data.astype(float) # needed for FFT code to work properly
    
    # Apply Hann window to the left edge
    left_window = np.hanning(2 * taper_length)[:taper_length]
    windowed_data[:taper_length] *= left_window
    
    # Apply Hann window to the right edge
    right_window = np.hanning(2 * taper_length)[taper_length:]
    windowed_data[-taper_length:] *= right_window
    
    return windowed_data

#print(type(apply_hann_window1(df, "NS_Normalized_Shifted")))

##### Coherent gain equation:
$G_c = \frac{1}{N}\sum_{n=0}^{N-1}\omega[n]$

- $G_c$: coherent gain
- $\omega[n]$: window
- $N$: window length

In [ ]:
def calculate_coherent_gain(data, taper_fraction=0.1):
    """Calculate coherent gain for partial Hann taper"""
    n=len(data)
    taper_length = int(n * taper_fraction)
    
    # Create the full window
    window = np.ones(n)
    
    # Left taper
    left_window = np.hanning(2 * taper_length)[:taper_length]
    window[:taper_length] = left_window
    
    # Right taper
    right_window = np.hanning(2 * taper_length)[taper_length:]
    window[-taper_length:] = right_window
    
    # Coherent gain is the mean
    coherent_gain = np.mean(window)
    
    return coherent_gain

# For taper_fraction = 0.1
coherent_gain = calculate_coherent_gain(df['NS_Normalized_Shifted'], taper_fraction=0.1)
#print(coherent_gain)

#### Approach 1: Overlay FFT results for tapered/subsampled data onto single figure with full DNS results
- plot for each dataframe as needed
    - may need to convert subsampled data from arrays to dataframes

In [ ]:
# smoothing option: moving average
# note to self: check
def smooth_spectrum(magnitudes, window_size=5):
    """Apply moving average smoothing to FFT magnitudes
    magnitudes: array of FFT magnitude values
    window_size: number of points to average (must be odd for symmetry)"""
    return np.convolve(magnitudes, np.ones(window_size)/window_size, mode='same')

In [ ]:
# convert subsampled data from arrays to DataFrames:
NS_sub_results_dframe = pd.DataFrame(NS_sub_results_df, columns=["NS_subsampled"])
EW_sub_results_dframe = pd.DataFrame(EW_sub_results_df, columns=["EW_subsampled"])
Z_sub_results_dframe = pd.DataFrame(Z_sub_results_df, columns=["Z_subsampled"])

# Plotting FFT spectra showing tapered/subsampled data overlayed onto full DNS data for a given dataframe
def plot_fft(data, column, sr, fig1=None, fig2=None, name='', apply_window=True, smooth=False, smooth_window=5):
    """Plot FFT with windowing correction.
    data: dataframe of interest
    column: column of interest from data
    name: name of data column, string
    sr: Sampling rate
    apply_window: bool, says whether to apply Hann window (default True)
    smooth: bool, says whether to apply moving average smoothing
    smooth_window: size of smoothing window"""
    n = len(data)
    
    # Apply window if requested
    if apply_window:
        windowed_data = apply_hann_window1(data, column, taper_fraction=0.1)
        # Calculate coherent gain for partial taper
        coherent_gain = calculate_coherent_gain(data, taper_fraction=0.1)
    else:
        windowed_data = data[column].values
        coherent_gain = 1.0
    
    # Compute FFT
    fft_values = scipy.fft.rfft(windowed_data)
    freqs = scipy.fft.rfftfreq(n, d=1/sr)
    
    # magnitudes WITH coherent gain correction
    magnitudes = np.abs(fft_values) / (n * coherent_gain)
    
    # Double the AC components (not DC and Nyquist)
    magnitudes[1:] *= 2.0
    
    # Undo doubling for Nyquist if even length
    if n % 2 == 0:
        magnitudes[-1] /= 2.0

    #phases= np.angle(fft_values) leaving out phases for now
    
    # Apply smoothing if requested
    if smooth:
        magnitudes = smooth_spectrum(magnitudes, window_size=smooth_window)
    
    # Create figures:
    if fig1 is None:
        fig1 = plt.figure(figsize=(10,6))
        plt.plot(freqs, magnitudes, linewidth=1, label=name)
        plt.title('FFT Magnitude Spectrum (Linear)', fontsize=16)
        plt.xlabel('Frequency (Hz)', fontsize=14)
        plt.ylabel('Magnitude', fontsize=14)
        plt.grid(True, which='major', alpha=0.5)
        plt.grid(True, which='minor', alpha=0.5)
        plt.minorticks_on()
        plt.xticks(fontsize=14)
        plt.yticks(fontsize=14)
        plt.tight_layout()
        if name:
            plt.legend(loc='upper right')
    else:
        plt.figure(fig1.number)
        plt.plot(freqs, magnitudes, linewidth=1, label=name)
        if name:
            plt.legend(loc='upper right')
    
    if fig2 is None:
        fig2 = plt.figure(figsize=(10, 6))
        plt.loglog(freqs[1:], magnitudes[1:], linewidth=1, label=name)
        plt.title('FFT Magnitude Spectrum (log)', fontsize=16)
        plt.xlabel('Frequency (Hz)', fontsize=14)
        plt.ylabel('Magnitude', fontsize=14)
        plt.xticks(fontsize=14)
        plt.yticks(fontsize=14)
        plt.grid(True, which='major', alpha=0.5)
        plt.grid(True, which='minor', alpha=0.5)
        plt.tight_layout()
        if name:
            plt.legend(loc='lower left')
    else:
        plt.figure(fig2.number)
        plt.loglog(freqs[1:], magnitudes[1:], linewidth=1, label=name)
        plt.legend()
        if name:
            plt.legend(loc='lower left')
    
    # if fig3 is None:
    #     fig3 = plt.figure(figsize=(10, 6))
    #     plt.plot(freqs[1:], phases[1:], linewidth=1, label=name)
    #     plt.title('Phase Spectrum', fontsize=16)
    #     plt.xlabel('Frequency (Hz)', fontsize=14)
    #     plt.ylabel('Phase (radians)', fontsize=14)
    #     plt.grid(True, alpha=0.5)
    #     plt.xticks(fontsize=14)
    #     plt.yticks(fontsize=14)
    #     #plt.xlim(-1, 50)
    #     plt.tight_layout()
    #     if name:
    #         plt.legend(loc='lower right', framealpha=0.5)
    # else:
    #     plt.figure(fig3.number)
    #     plt.plot(freqs[1:], phases[1:], linewidth=1, label=name)
    #     if name:
    #         plt.legend(loc='lower right', framealpha=0.8)
    
    return fig1, fig2#, fig3 
#####################################################################################################################
#Use to create figures for one dataframe at a time:
print("dataframe: df")
#North-South for df:
# For full NS data
fig1, fig2 = plot_fft(df, 'NS_Normalized_Shifted', sr=original_sampling_rate, name='Full NS Data', apply_window=False, smooth=True, smooth_window=5)
# For tapered/subsampled data 
plot_fft(NS_sub_results_dframe, 'NS_subsampled', new_sr_df, fig1=fig1, fig2=fig2, name='Subsampled/Tapered NS Data', apply_window=True,
         smooth=True, smooth_window=5)
plt.show()

#East-West:
# For full EW data
fig1, fig2 = plot_fft(df, 'EW_Normalized_Shifted', sr=original_sampling_rate, name='Full EW Data', apply_window=False, smooth=True, smooth_window=5)
# For tapered/subsampled data 
plot_fft(EW_sub_results_dframe, 'EW_subsampled', new_sr_df, fig1=fig1, fig2=fig2, name='Subsampled/Tapered EW Data', apply_window=True, 
         smooth=True, smooth_window=5)
plt.show()

#Up-Down:
# For full Z data
fig1, fig2 = plot_fft(df, 'Z_Normalized_Shifted', sr=original_sampling_rate, name='Full Z Data', apply_window=False, smooth=True, smooth_window=5)
# For tapered/subsampled data 
plot_fft(Z_sub_results_dframe,'Z_subsampled', new_sr_df, fig1=fig1, fig2=fig2, name='Subsampled/Tapered Z Data',
         apply_window=True, smooth=True, smooth_window=5)
plt.show()

### Horizontal and Vertical FFTs
- Goal: Add NS and EW FFTs together to get one set of plots (Horizontal)
- Leave Z FFT plots as is (Vertical), plotted from a previous cell

#### Horizontal FFT Plots:

In [ ]:
# overlays tapered/subsampled result onto full DNS result for given dataframe
def plot_fft_horizontal(data_ns, column_ns, data_ew, column_ew, sr, fig1=None, fig2=None, name='', apply_window=True, smooth=False, smooth_window=5):
    """Plot horizontal FFT spectrum from North-South and East-West components.
    data_ns: North-South dataframe with column: column_ns
    data_ew: East-West dataframe with column: column_ew
    sr: Sampling rate
    name: data label for legend, string
    apply_window: whether to apply hann window to input data, bool
    smooth: whether to apply moving average smoothing
    smooth_window: size of smoothing window"""
    n = len(data_ns)
    
    # Apply window if requested
    if apply_window:
        windowed_data_ns = apply_hann_window1(data_ns, column_ns, taper_fraction=0.1)
        windowed_data_ew = apply_hann_window1(data_ew, column_ew, taper_fraction=0.1)
        
        # Calculate coherent gain for partial taper:
        coherent_gain = calculate_coherent_gain(data_ns[column_ns], taper_fraction=0.1)
    else:
        windowed_data_ns = data_ns[column_ns].values
        windowed_data_ew = data_ew[column_ew].values
        coherent_gain = 1.0
    
    # Compute FFT for both components
    fft_ns = scipy.fft.rfft(windowed_data_ns)
    fft_ew = scipy.fft.rfft(windowed_data_ew)
    freqs = scipy.fft.rfftfreq(n, d=1/sr)
    
    # Compute magnitudes WITHOUT coherent gain correction
    mag_ns = np.abs(fft_ns) / n
    mag_ew = np.abs(fft_ew) / n
    
    # Double the AC components (not DC and Nyquist)
    mag_ns[1:] *= 2.0
    mag_ew[1:] *= 2.0
    
    # Undo doubling for Nyquist if even length
    if n % 2 == 0:
        mag_ns[-1] /= 2.0
        mag_ew[-1] /= 2.0
    
    # Combine as RMS for horizontal magnitude?
    #magnitudes = np.sqrt((mag_ns**2 + mag_ew**2) / 2)
    # Combine horizontal magnitudes with vector magnitude formula:
    magnitudes = np.sqrt((mag_ns**2) + (mag_ew**2))
    # or change to preferred method of your choosing

    # Phases:
    # phases_ns = np.angle(fft_ns) 
    # phases_ew = np.angle(fft_ew)

    # Apply smoothing if requested:
    if smooth:
        magnitudes = smooth_spectrum(magnitudes, window_size=smooth_window)
    
    # Create figures:
    if fig1 is None:
        fig1 = plt.figure(figsize=(10,6))
        plt.plot(freqs, magnitudes, linewidth=1, label=name)
        plt.title('FFT Magnitude Spectrum (Linear)', fontsize=16)
        plt.xlabel('Frequency (Hz)', fontsize=14)
        plt.ylabel('Magnitude', fontsize=14)
        plt.grid(True, which='major', alpha=0.5)
        plt.grid(True, which='minor', alpha=0.5)
        plt.minorticks_on()
        plt.xticks(fontsize=14)
        plt.yticks(fontsize=14)
        plt.tight_layout()
        if name:
            plt.legend(loc='upper right')
    else:
        plt.figure(fig1.number)
        plt.plot(freqs, magnitudes, linewidth=1, label=name)
        if name:
            plt.legend(loc='upper right')
    
    if fig2 is None:
        fig2 = plt.figure(figsize=(10, 6))
        plt.loglog(freqs[1:], magnitudes[1:], linewidth=1, label=name)
        plt.title('FFT Magnitude Spectrum (log)', fontsize=16)
        plt.xlabel('Frequency (Hz)', fontsize=14)
        plt.ylabel('Magnitude', fontsize=14)
        plt.xticks(fontsize=14)
        plt.yticks(fontsize=14)
        plt.grid(True, which='major', alpha=0.5)
        plt.grid(True, which='minor', alpha=0.5)
        plt.tight_layout()
        if name:
            plt.legend(loc='lower left')
    else:
        plt.figure(fig2.number)
        plt.loglog(freqs[1:], magnitudes[1:], linewidth=1, label=name)
        plt.legend()
        if name:
            plt.legend(loc='lower left')
    
    #if fig3 is None:
     #   fig3 = plt.figure(figsize=(10, 6))
      #  plt.plot(freqs[1:], phases[1:], linewidth=1, label=name)
       # plt.title('Phase Spectrum', fontsize=16)
        #plt.xlabel('Frequency (Hz)', fontsize=14)
        #plt.ylabel('Phase (radians)', fontsize=14)
        #plt.grid(True, alpha=0.5)
        #plt.xticks(fontsize=14)
        #plt.yticks(fontsize=14)
        #plt.xlim(-1, 50)
        #plt.tight_layout()
        #if name:
         #   plt.legend(loc='lower right', framealpha=0.5)
    #else:
     #   plt.figure(fig3.number)
      #  plt.plot(freqs[1:], phases[1:], linewidth=1, label=name)
       # if name:
        #    plt.legend(loc='lower right', framealpha=0.8)
    
    return freqs, magnitudes, fig1, fig2#, fig3   Leaving phases out for now

print("horizontal FFT for dataframe: df")
# First call: Full data - creates new figures
freqs_full, mags_full, fig1, fig2 = plot_fft_horizontal(df, 'NS_Normalized_Shifted', df, 'EW_Normalized_Shifted', 
    sr=original_sampling_rate, name='Full Horizontal Data', apply_window=False, smooth=True, smooth_window=5)

# Second call: Subsampled/tapered data - reuses the same figures
NS_sub_results_df_dframe = pd.DataFrame(NS_sub_results_df, columns=['Subsampled NS from df'])
EW_sub_results_df_dframe = pd.DataFrame(EW_sub_results_df, columns=['Subsampled EW from df'])
freqs_sub, mags_sub, fig1, fig2 = plot_fft_horizontal(NS_sub_results_df_dframe, 'Subsampled NS from df', 
                                EW_sub_results_df_dframe, 'Subsampled EW from df', sr=new_sr_df,
    fig1=fig1,  # Pass the existing figures
    fig2=fig2, name='Subsampled/Tapered Horizontal Data', apply_window=True, smooth=True, smooth_window=5)

plt.show()

***
### FFT With Moving Window
##### Notes:
- Goal: apply moving window (20 second intervals)
- Using short time fourier transform from scipy.signal
- Issue with previous version of code: time was not the same dimensions as the STFT result
    - Possible solutions: Plot spectrogram to show frequency content overtime, or plot average magnitude spectrum
- Tried log scale for spectrogram frequency axis, did not display well for most full data plots
- adjust y-axis limits to focus on areas of interest

#### Spectrograms

In [ ]:
# ShortTimeFFT(win, hop, fs, *, fft_mode='onesided', mfft=None, dual_win=None, scale_to=None, phase_shift=0)
# win = window function
# hop = time increment in signal samples for sliding window
# fs = sampling frequency of output signal and window
# fft_mode = Mode of FFT to be used (default ‘onesided’)
# mfft = Length of the FFT used, if a zero padded FFT is desired. If None (default), the length of the window win is used

# Problem with previous code (deleted): time not same dimensions as STFT result
# Possible solutions:
# Method 1: spectrogram
def hann_windowed_fft_spectrogram(data, sr, plot_title, vmax, vmin, window_duration=20):
    """Compute FFT with scipy short time fourier transform
    data: Time series to perform STFT on (array)
    sr: sampling rate
    plot_title: title for spectrogram plot (string)
    window_duration: window duration in seconds"""
    hop = int(sr*window_duration) # hop = number of samples in window_duration (20sec)
    
    # Generate hann window array with tapering only at edges
    window_array = np.ones(hop)  # Start with flat window
    taper_length = int(hop * 0.1)  # 0.1 for tapering at 10% from edges
    
    # Apply taper to left edge
    left_window = np.hanning(2*taper_length)[:taper_length]
    window_array[:taper_length] = left_window
    
    # Apply taper to right edge
    right_window = np.hanning(2*taper_length)[taper_length:]
    window_array[-taper_length:] = right_window
    
    STF = ShortTimeFFT(win=window_array, hop=hop, fs=sr, fft_mode='onesided', mfft=None, scale_to='magnitude', phase_shift=0)
    STF_result = STF.stft(data)
    
    # Get time and frequency arrays
    time_axis = STF.t(len(data))  # Time values for each window (sec)
    time_minute = time_axis/60
    freq_axis = STF.f  # Frequency values
    
    # Plot spectrogram
    stft_plot = plt.figure(figsize=(12,4))
    plt.pcolormesh(time_minute, freq_axis, np.abs(STF_result), cmap='binary', shading='gouraud', vmax=vmax, vmin=vmin)
    plt.title(plot_title)
    plt.xlabel("Time (min)")
    plt.ylabel("Frequency (Hz)")
    #plt.yscale('log')
    cbar=plt.colorbar(label='Magnitude')
    cbar.formatter.set_powerlimits((0, 0))
    plt.grid(which='major')
    plt.grid(which='minor')
    
    return stft_plot

for i, dframe in enumerate(dataframes):
    if i==0:
        print("dataframe: df")
    else:
        print(f"dataframe: df{i+1}")
    # Plot!------------------------------------------------------------
    # Spectrograms for NS data:
    print("Spectrograms for North-South data")
    # Full NS data:
    hann_windowed_fft_spectrogram(dframe['NS_Normalized_Shifted'].values, sr=original_sampling_rate, 
                             plot_title='Short Time Fourier Transform With 20s Hann Window (Full NS Data)', vmax=0.001, vmin=0, window_duration=20)
    plt.ylim(top=500)
    plt.tight_layout()
    plt.show()
    # Tapered/subsampled NS data:
    hann_windowed_fft_spectrogram(NS_taper_sub[i], sr=new_sr_df, plot_title='Short Time Fourier Transform With 20s Hann Window (Tapered/Subsampled NS Data)',
                             vmax=0.001, vmin=0, window_duration=20) 
    # new sampling rate should be the same for all dataframes because they have been subsampled the same
    plt.ylim(top=70)
    plt.tight_layout()
    plt.show()
    #---------------------------------------------------------------------------------
    # Spectrograms for EW data:
    print("Spectrograms for East-West data")
    # Full EW data:
    hann_windowed_fft_spectrogram(dframe['EW_Normalized_Shifted'].values, sr=original_sampling_rate, 
                             plot_title='Short Time Fourier Transform With 20s Hann Window (Full EW Data)', vmax=0.001, vmin=0, window_duration=20)
    plt.ylim(top=500)
    plt.tight_layout()
    plt.show()
    # Tapered/subsampled EW data:
    hann_windowed_fft_spectrogram(EW_taper_sub[i], sr=new_sr_df, plot_title='Short Time Fourier Transform With 20s Hann Window (Tapered/Subsampled EW Data)',
                             vmax=0.001, vmin=0, window_duration=20)
    plt.ylim(top=70)
    plt.tight_layout()
    plt.show()
    #--------------------------------------------------------------------------------
    # Spectrograms for Z-component data
    print("Spectrograms for Z-component data")
    # Full Z data:
    hann_windowed_fft_spectrogram(dframe['Z_Normalized_Shifted'].values, sr=original_sampling_rate, 
                             plot_title='Short Time Fourier Transform With 20s Hann Window (Full Z Data)', vmax=0.001, vmin=0, window_duration=20)
    plt.ylim(top=500)
    plt.tight_layout()
    plt.show()
    # Tapered/subsampled Z data:
    hann_windowed_fft_spectrogram(Z_taper_sub[i], sr=new_sr_df, plot_title='Short Time Fourier Transform With 20s Hann Window (Tapered/Subsampled Z Data)',
                             vmax=0.001, vmin=0, window_duration=20)
    plt.ylim(top=70)
    plt.tight_layout()
    plt.show()
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

***
### H/V Ratio
#### Notes:
- Divide horizontal magnitude by vertical magnitude to get H/V ratio
- Used vector-magnitude formula to combine horizontal magnitudes for North-South and East-West components

#### Filter for Smoothing
- applied Savitzky-Golay filter for smoothing data
    - also tried an envelope function for smoothing using the Hilbert transform
- limited displayed frequency range

In [ ]:
## Adding envelope for smoothing using Hilbert transform:
def envelope(input_data):
    """Compute envelope of input_data using Hilbert transform"""
    signal= hilbert(input_data)
    return np.abs(signal)

In [ ]:
# original w updated tapering function:
# (Note: this version is shown in your updated slides)
def HV_ratio(data_ns, column_ns, data_ew, column_ew, data_z, column_z, sr, apply_window=True, freq_min=0.1, freq_max=None):
    """Compute Horizontal-to-Vertical Spectral Ratio (HVSR)
    data_ns: North-South component (array)
    data_ew: East-West component (array)
    data_z: Vertical (Z) component (array)
    sr: Sampling rate
    freq_min: Minimum frequency to include (Hz)
    freq_max: Maximum frequency to include (Hz), default is 80% of Nyquist"""
    n = len(data_ns)
    
    if freq_max is None:
        freq_max = 0.8*(sr/2)  # 80% of Nyquist to avoid edge effects
    
    # Apply window if requested
    if apply_window:
        windowed_data_ns = apply_hann_window1(data_ns, column_ns, taper_fraction=0.1).values
        windowed_data_ew = apply_hann_window1(data_ew, column_ew, taper_fraction=0.1).values
        windowed_data_z = apply_hann_window1(data_z, column_z, taper_fraction=0.1).values
        
        coherent_gain = calculate_coherent_gain(data_ns[column_ns], taper_fraction=0.1)
    else:
        windowed_data_ns = data_ns[column_ns].values
        windowed_data_ew = data_ew[column_ew].values
        windowed_data_z = data_z[column_z].values
        coherent_gain = 1.0
    
    # Compute FFTs
    fft_ns = np.fft.rfft(windowed_data_ns)
    fft_ew = np.fft.rfft(windowed_data_ew)
    fft_z = np.fft.rfft(windowed_data_z)
    
    freqs = np.fft.rfftfreq(n, d=1/sr)
    
    # Compute magnitudes
    mag_ns = np.abs(fft_ns) / n
    mag_ew = np.abs(fft_ew) / n
    mag_z = np.abs(fft_z) / n
    
    # Double AC components for one-sided spectrum correction
    mag_ns[1:] *= 2.0
    mag_ew[1:] *= 2.0
    mag_z[1:] *= 2.0
    
    if n % 2 == 0:
        mag_ns[-1] /= 2.0
        mag_ew[-1] /= 2.0
        mag_z[-1] /= 2.0
    
    # Compute horizontal RMS
    # mag_horizontal = np.sqrt((mag_ns**2 + mag_ew**2) / 2)
    # Use vector-magnitude formula to combine horizontal magnitudes:
    mag_horizontal = np.sqrt((mag_ns**2) + (mag_ew**2))
    
    # Compute HVSR with threshold to avoid division by tiny numbers
    z_threshold = np.max(mag_z) * 1e-8
    hvsr = np.zeros_like(mag_horizontal)
    mask = mag_z > z_threshold
    hvsr[mask] = mag_horizontal[mask] / mag_z[mask]
    
    # Clip extreme values?
    #hvsr = np.clip(hvsr, 0, clip_max) # comment out when don't want to use
    
    # Apply frequency range mask
    freq_mask = (freqs >= freq_min) & (freqs <= freq_max)
    
    return freqs[freq_mask], hvsr[freq_mask]

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
print("dataframe: df")
# Compute HVSR for full data (df)
freqs1, hvsr1 = HV_ratio(df, 'NS_Normalized_Shifted', df, 'EW_Normalized_Shifted', df, 'Z_Normalized_Shifted',
        sr=original_sampling_rate, apply_window=False, freq_min=0.1, freq_max=300)  # Or use None for auto freq_max
        #clip_max=10)

# Compute HVSR for tapered/subsampled data (df)
Z_sub_results_df_dframe = pd.DataFrame(Z_sub_results_df, columns=['Subsampled Z from df'])
freqs2, hvsr2 = HV_ratio(NS_sub_results_df_dframe, 'Subsampled NS from df', EW_sub_results_df_dframe, 'Subsampled EW from df',
        Z_sub_results_df_dframe, 'Subsampled Z from df', sr=new_sr_df, apply_window=True, freq_min=0.1,
        freq_max=70)  # Should be less than decimated Nyquist (~73 Hz for subsampling step=7)
        #clip_max=10)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Apply smoothing with Savitzky-Golay filter
hvsr1_smooth = savgol_filter(hvsr1, window_length=20481, polyorder=2) ## For 20s window, 20s*(sampling rate in /s) = number of samples in 20 sec
# Note: window length should be an odd integer to allow the filter window to be centered symmetrically on the current data point
hvsr2_smooth = savgol_filter(hvsr2, window_length=2925, polyorder=2) # 20*new_sr_df = approx. 2925
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Try second pass through Savitzky-Golay filter instead of envelope:
hvsr1_smooth = savgol_filter(hvsr1_smooth, window_length=20001, polyorder=2)
hvsr2_smooth = savgol_filter(hvsr2_smooth, window_length=10001, polyorder=2)
# third pass?
hvsr1_smooth = savgol_filter(hvsr1_smooth, window_length=10001, polyorder=2)
hvsr2_smooth = savgol_filter(hvsr2_smooth, window_length=10001, polyorder=2)
hvsr2_smooth = savgol_filter(hvsr2_smooth, window_length=10001, polyorder=2) # 4th pass for tapered/subsampled data
hvsr2_smooth = savgol_filter(hvsr2_smooth, window_length=10001, polyorder=2) # 5th pass for tapered/subsampled data

########## Testing envelope with Hilbert transform ################################################################################
# call envelope function from previous cell:
#hvsr1_smooth = envelope(hvsr1_smooth)
#hvsr2_smooth = envelope(hvsr2_smooth)
# doesn't work well, adds spikes at edges of plot
###################################################################################################################################################
# Find main peak (full data)
peak_idx = np.argmax(hvsr1_smooth[(freqs1 > 10) & (freqs1 < 50)])
peak_freq = freqs1[(freqs1 > 10) & (freqs1 < 50)][peak_idx]
peak_value = hvsr1_smooth[(freqs1 > 10) & (freqs1 < 50)][peak_idx]
print(f"Full data peak frequency: {peak_freq} Hz")
print(f"Full data peak H/V: {peak_value}")
# (tapered/subsampled data):
peak_idx2 = np.argmax(hvsr2_smooth[(freqs2 > 10) & (freqs2 < 50)])
peak_freq2 = freqs2[(freqs2 > 10) & (freqs2 < 50)][peak_idx2]
peak_value2 = hvsr2_smooth[(freqs2 > 10) & (freqs2 < 50)][peak_idx2]
print(f"Tapered/Subsampled data peak frequency: {peak_freq2} Hz")
print(f"Tapered/Subsampled data peak H/V: {peak_value2}")

# Plot
plt.figure(figsize=(12, 6))
# Smoothed data
plt.semilogx(freqs1, hvsr1_smooth, label="H/V From Full Data")
plt.semilogx(freqs2, hvsr2_smooth, label="H/V From Tapered/Subsampled Data")
plt.axhline(y=1, color='grey', linestyle='--', alpha=0.5, label='H/V = 1')
plt.xlabel('Frequency (Hz)')
plt.ylabel('H/V Ratio')
plt.title('Smoothed Horizontal-to-Vertical Spectral Ratio (HVSR) - Ver.1')
plt.ylim([0, 5])  # Adjust based on your data
plt.grid(True, which="both", alpha=0.5)
plt.axvline(x=30.63, linestyle="--", color='green', label='30.63 Hz')
plt.legend(loc='upper left')
plt.show()
print("Note: 30.63 +/- 3.81 Hz is the max. H/V from screenshot of Grilla average H/V plot")

#### Interpreting Horizontal-to-Vertical Spectral Ratios
- Peaks should occur at fundamental and higher order resonance frequencies
    - (fundamental frequency = lowest resonant frequency in a system)
- Typically, the fundamental resonance frequency is the first/lowest value indicated by the highest amplitude peak on the H/V frequency spectrum
- H/V = 1 indicates that the horizontal and vertical motions are equal
- H/V < 1 indicates that horizontal motion is weaker than vertical motion
- H/V > 1 indicates that horizontal motion is stronger than vertical motion

### Plotting H/V Ratio From Grilla .txt File:

In [ ]:
# Trying to use H/V .txt file downloaded from Grilla:
column_names = ['freq.', 'H/V']
dtf = pd.read_table('Tyendinaga_HV_ratio.asc', delimiter=r'\s+', encoding='latin-1', skiprows=31, names=column_names, usecols=column_names)
#dtf.head().style

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(dtf['freq.'], dtf['H/V'], label="H/V Ratio from Grilla Data File")
plt.xlabel("Frequency (Hz)")
plt.ylabel("H/V Ratio")
plt.title("H/V Ratio from Grilla Data File")
plt.axvline(x=30.63, color='red', linestyle='--', label="30.63 Hz")
plt.legend()
plt.grid(True, alpha=0.5)
plt.xscale('log')
plt.show()

In [ ]:
# Remove unneeded columns: NS, EW, Z, nsL, ewL, zL, aY, aX, aZ, NS_Detrended, EW_Detrended, Z_Detrended, NS_Normalized, EW_Normalized, Z_Normalized
# Will be left with DataFrame containing time (s) and the Normalized_Shifted data for NS, EW, and Z components
df_updated = df.drop(['NS', 'EW', 'Z', 'nsL', 'ewL', 'zL', 'aY', 'aX', 'aZ', 'NS_Detrended', 'EW_Detrended', 'Z_Detrended', 'NS_Normalized', 
                      'EW_Normalized', 'Z_Normalized'], axis=1)
# df_updated.head()

### Convert Cell Outputs to PDF:
- can currently download as .html file, which when opened in a browser can be saved as a pdf using print (Ctrl+P) -> save to pdf <br>
    - using code to directly download as .pdf file is not working currently
    - if opened with Jupyter Notebook, can save full file as .pdf directly by going to File -> Save and Export Notebook As -> PDF

In [ ]:
# Save notebook as .html file
def save_notebook(notebook_name, output_name=None):
    """Save Jupyter notebook outputs and markdown to .html file (excludes code cells)
    notebook_name : str, name of the notebook file (e.g., 'notebook.ipynb')
    output_name : str, name for the output file (optional). If None, uses the notebook name with .html extension"""
    
    # Set output file name
    if output_name is None:
        output_name = notebook_name.replace('.ipynb', '.html')
    
    # Check if notebook file exists
    if not os.path.exists(notebook_name):
        raise FileNotFoundError(f"Notebook file '{notebook_name}' not found")
            
    result = subprocess.run(['jupyter', 'nbconvert', '--to', 'html', '--no-input',  # Exclude code cells
    notebook_name, '--output', output_name.replace('.html', '')], capture_output=True, text=True)
            
    if result.returncode == 0: # return code of zero indicates successful execution 
        print(f"Created HTML file: {output_name}")
        print("Open this in your browser and use Ctrl+P --> Save as PDF")
        print("(The HTML file contains only outputs and markdown, no code cells)")
    else:
        print("Error:")
        print(result.stderr) # standard error

# Example usage:
# save_notebook('notebook.ipynb')
# Or specify custom output name:
# save_notebook('notebook.ipynb', 'output_notebook.html')
save_notebook('Proj_2_File_1_Tyendinaga_Updated-Copy1.ipynb', 'Proj_2_File_1_Tyendinaga_Updated-Copy1.html')